In [1]:
import os
from saga_gwrd import gwrdownxcale
outdir = "/home/ljp238/Downloads/SAGA_DEV/"

Plan:
- Svar s and e
- Mvar s and e 

this [artitcle](https://acolita.com/aumentar-la-resolucion-de-un-raster-usando-la-tecnica-downscaling-en-qgis/) does 121 mapping, so we might be justified in doing Svar alone

##### S-VAR

In [2]:
name = 'GWRd_svs'
ypath = "/media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N13E103/N13E103_gdtm_riofill0_egm.tif"
#/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N13E103/N13E103_gedilow.tif
geoid_fn = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N13E103/N13E103_egm08.tif"

xpath = "/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N13E103/N13E103_edem_egm.tif"
#/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N13E103/N13E103_s1.tif
#/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N13E103/N13E103_tdem_hem.tif


In [3]:
tname = xpath.split('/')[-2]
opath =  os.path.join(outdir,f'{tname}_{name}.tif')

In [4]:
outpaths = gwrdownxcale(xpath, ypath, opath,geoid_fn, 
                        oaux=False, epsg_code=4979, clean=True,
                        search_range=0, search_radius=10, dw_weighting=0, dw_idw_power=2.0, 
                        dw_bandwidth=1.0,logistic=0, model_out=0, grid_system=None)

____________________________

   #####   ##   #####    ##
  ###     ###  ##       ###
   ###   # ## ##  #### # ##
    ### ##### ##    # #####
 ##### #   ##  ##### #   ##
____________________________

SAGA Version: 8.2.2

____________________________
library path: /usr/lib/x86_64-linux-gnu/saga/
library name: libstatistics_regression
library     : statistics_regression
tool        : GWR for Grid Downscaling
identifier  : 14
author      : O.Conrad (c) 2013
processors  : 56 [56]
____________________________

loading: N13E103_edem_egm

100%
loading: N13E103_gdtm_riofill0_egm

 99%
[GWR for Grid Downscaling] Execution started...

__________
[GWR for Grid Downscaling] Parameters:

Grid System: 0.000111; 9001x 9001y; 103x 13y
Predictors: 1 object (N13E103_edem_egm)
Regression: Regression
Regression with Residual Correction: <not set>
Dependent Variable: N13E103_gdtm_riofill0_egm
Coefficient of Determination: Coefficient of Determination
Residuals: Residuals
Regression Parameters: No objects
L

Segmentation fault (core dumped)


! The file "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw0.tif" already exists.
GWR Grid Downscaling completed.
Cleaning up intermediate files...
/home/ljp238/Downloads/SAGA_DEV
fmin_fn...
bcor_fn...
fminbcor_fn...
Start time: 2025-05-15 13:11:08
End time: 2025-05-15 13:12:43
RUN.TIME = 1 minutes, 34 seconds


In [4]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

def resample_raster(src_path, match_path, resampling=Resampling.bilinear):
    with rasterio.open(match_path) as match_ds:
        match_transform = match_ds.transform
        match_crs = match_ds.crs
        match_width = match_ds.width
        match_height = match_ds.height

        with rasterio.open(src_path) as src_ds:
            data = np.empty((src_ds.count, match_height, match_width), dtype=np.float32)

            for i in range(src_ds.count):
                reproject(
                    source=rasterio.band(src_ds, i + 1),
                    destination=data[i],
                    src_transform=src_ds.transform,
                    src_crs=src_ds.crs,
                    dst_transform=match_transform,
                    dst_crs=match_crs,
                    resampling=resampling
                )

            profile = match_ds.profile.copy()
            profile.update({
                'height': match_height,
                'width': match_width,
                'transform': match_transform,
                'dtype': 'float32'
            })

            return data, profile

def bcor_sub(fine_path, coarse_path, output_path):
    coarse_resampled, profile = resample_raster(coarse_path, fine_path)

    with rasterio.open(fine_path) as fine_ds:
        fine_data = fine_ds.read().astype(np.float32)

    # Subtract (fine - coarse)
    diff = fine_data - coarse_resampled

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(diff)

def fmin_get(raster_a_path, raster_b_path, output_path):
    """
    Create a new raster where each pixel is the minimum of the corresponding pixels in two input rasters.
    Nodata values are treated as np.nan.
    
    Parameters:
        raster_a_path (str): File path to the first input raster.
        raster_b_path (str): File path to the second input raster.
        output_path (str): File path to save the output raster.
    """
    # Open raster a
    with rasterio.open(raster_a_path) as src_a:
        a = src_a.read(1).astype(float)
        a[a == src_a.nodata] = np.nan
        profile = src_a.profile.copy()

    # Open raster b
    with rasterio.open(raster_b_path) as src_b:
        b = src_b.read(1).astype(float)
        b[b == src_b.nodata] = np.nan

    # Compute pixel-wise minimum, treating np.nan properly
    c = np.fmin(a, b)

    # Set a float nodata value if needed (optional)
    nodata_value = -9999.0
    c[np.isnan(c)] = nodata_value
    profile.update(dtype='float32', nodata=nodata_value)

    # Write output raster
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(c.astype('float32'), 1)

In [8]:
gwrp_fn = gwrdownxcale(xpath, ypath, opath, oaux=False, epsg_code=4979, clean=True,
                 search_range=0, search_radius=10, dw_weighting=0, dw_idw_power=2.0, dw_bandwidth=1.0,
                 logistic=0, model_out=0, grid_system=None)

fmin_fn = gwrp_fn.replace('.tif', '_fmin.tif')
fmin_get(xpath, gwrp_fn, fmin_fn) # xpath must be tdx 

bcor_fn = gwrp_fn.replace('.tif', "_bcor.tif")
bcor_sub(gwrp_fn, geoid_fn, bcor_fn)


fminbcor_fn = bcor_fn.replace('.tif', '_fmin.tif')
fmin_get(xpath, bcor_fn, fminbcor_fn) # xpath must be tdx 

____________________________

   #####   ##   #####    ##
  ###     ###  ##       ###
   ###   # ## ##  #### # ##
    ### ##### ##    # #####
 ##### #   ##  ##### #   ##
____________________________

SAGA Version: 8.2.2

____________________________
library path: /usr/lib/x86_64-linux-gnu/saga/
library name: libstatistics_regression
library     : statistics_regression
tool        : GWR for Grid Downscaling
identifier  : 14
author      : O.Conrad (c) 2013
processors  : 56 [56]
____________________________

loading: N13E103_edem_egm

100%
loading: N13E103_gdtm_riofill0_egm

 99%
[GWR for Grid Downscaling] Execution started...

__________
[GWR for Grid Downscaling] Parameters:

Grid System: 0.000111; 9001x 9001y; 103x 13y
Predictors: 1 object (N13E103_edem_egm)
Regression: Regression
Regression with Residual Correction: <not set>
Dependent Variable: N13E103_gdtm_riofill0_egm
Coefficient of Determination: Coefficient of Determination
Residuals: Residuals
Regression Parameters: No objects
L

Segmentation fault (core dumped)


! The file "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw0.tif" already exists.
GWR Grid Downscaling completed.
Cleaning up intermediate files...
/home/ljp238/Downloads/SAGA_DEV
Removing /home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw0_fmin.tif.aux.xml...
Removing /home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw0__bcor_fmin.tif.aux.xml...
Removing /home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw0__bcor.tif.aux.xml...
Start time: 2025-05-15 13:03:37
End time: 2025-05-15 13:04:38
RUN.TIME = 1 minutes, 0 seconds


In [ ]:
fmin_fn = gwrp_fn.replace('.tif', '_fmin.tif')
fmin_get(xpath, gwrp_fn, fmin_fn) # xpath must be tdx 

bcor_fn = gwrp_fn.replace('.tif', "_bcor.tif")
bcor_sub(gwrp_fn, geoid_fn, bcor_fn)


fminbcor_fn = bcor_fn.replace('.tif', '_fmin.tif')
fmin_get(xpath, bcor_fn, fminbcor_fn) # xpath must be tdx 
# it seems like i dont need to do Vtrasnfrom as i am using egm in xpath

In [ ]:
# why the fmin_get is not working properly like yesterday :: fix that [x]
# have lunch , and have this bit running for this tile as one pipeline
# do emails 
# time tasks as i need to the the application too 

In [ ]:
gwrdownxcale(xpath, ypath, opath, oaux=False, epsg_code=4979, clean=True,
                 search_range=0, search_radius=10, dw_weighting=1, dw_idw_power=2.0, dw_bandwidth=1.0,
                 logistic=0, model_out=0, grid_system=None)

bcor_fn = gwrp_fn.replace('.tif', "__bcor.tif")
bcor(gwrp_fn, geoid_fn, bcor_fn)

fmin_fn = bcor_fn.replace('.tif', '_fmin.tif')
fmin(xpath, bcor_fn, fmin_fn) # xpath must be tdx 

____________________________

   #####   ##   #####    ##
  ###     ###  ##       ###
   ###   # ## ##  #### # ##
    ### ##### ##    # #####
 ##### #   ##  ##### #   ##
____________________________

SAGA Version: 8.2.2

____________________________
library path: /usr/lib/x86_64-linux-gnu/saga/
library name: libstatistics_regression
library     : statistics_regression
tool        : GWR for Grid Downscaling
identifier  : 14
author      : O.Conrad (c) 2013
processors  : 56 [56]
____________________________

loading: N13E103_edem_egm

100%
loading: N13E103_gdtm_riofill0_egm

 99%
[GWR for Grid Downscaling] Execution started...

__________
[GWR for Grid Downscaling] Parameters:

Grid System: 0.000111; 9001x 9001y; 103x 13y
Predictors: 1 object (N13E103_edem_egm)
Regression: Regression
Regression with Residual Correction: <not set>
Dependent Variable: N13E103_gdtm_riofill0_egm
Coefficient of Determination: Coefficient of Determination
Residuals: Residuals
Regression Parameters: No objects
L

Segmentation fault (core dumped)


Input file size is 9001, 9001
0...10...20...30...40...50...60...70...80...90...100 - done.
# Successfully converted "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs.tif" to "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw1.tif".
GWR Grid Downscaling completed.
Cleaning up intermediate files...
/home/ljp238/Downloads/SAGA_DEV
Start time: 2025-05-15 11:41:10
End time: 2025-05-15 11:42:13
RUN.TIME = 1 minutes, 2 seconds


In [ ]:
gwrdownxcale(xpath, ypath, opath, oaux=False, epsg_code=4979, clean=True,
                 search_range=0, search_radius=10, dw_weighting=2, dw_idw_power=2.0, dw_bandwidth=1.0,
                 logistic=0, model_out=0, grid_system=None)

bcor_fn = gwrp_fn.replace('.tif', "__bcor.tif")
bcor(gwrp_fn, geoid_fn, bcor_fn)

fmin_fn = bcor_fn.replace('.tif', '_fmin.tif')
fmin(xpath, bcor_fn, fmin_fn) # xpath must be tdx 

____________________________

   #####   ##   #####    ##
  ###     ###  ##       ###
   ###   # ## ##  #### # ##
    ### ##### ##    # #####
 ##### #   ##  ##### #   ##
____________________________

SAGA Version: 8.2.2

____________________________
library path: /usr/lib/x86_64-linux-gnu/saga/
library name: libstatistics_regression
library     : statistics_regression
tool        : GWR for Grid Downscaling
identifier  : 14
author      : O.Conrad (c) 2013
processors  : 56 [56]
____________________________

loading: N13E103_edem_egm

100%
loading: N13E103_gdtm_riofill0_egm

 99%
[GWR for Grid Downscaling] Execution started...

__________
[GWR for Grid Downscaling] Parameters:

Grid System: 0.000111; 9001x 9001y; 103x 13y
Predictors: 1 object (N13E103_edem_egm)
Regression: Regression
Regression with Residual Correction: <not set>
Dependent Variable: N13E103_gdtm_riofill0_egm
Coefficient of Determination: Coefficient of Determination
Residuals: Residuals
Regression Parameters: No objects
L

Segmentation fault (core dumped)


Input file size is 9001, 9001
0...10...20...30...40...50...60...70...80...90...100 - done.
# Successfully converted "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs.tif" to "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw2.tif".
GWR Grid Downscaling completed.
Cleaning up intermediate files...
/home/ljp238/Downloads/SAGA_DEV
Start time: 2025-05-15 11:42:45
End time: 2025-05-15 11:43:48
RUN.TIME = 1 minutes, 3 seconds


In [ ]:
gwrdownxcale(xpath, ypath, opath, oaux=False, epsg_code=4979, clean=True,
                 search_range=0, search_radius=10, dw_weighting=3, dw_idw_power=2.0, dw_bandwidth=1.0,
                 logistic=0, model_out=0, grid_system=None)

bcor_fn = gwrp_fn.replace('.tif', "__bcor.tif")
bcor(gwrp_fn, geoid_fn, bcor_fn)

fmin_fn = bcor_fn.replace('.tif', '_fmin.tif')
fmin(xpath, bcor_fn, fmin_fn) # xpath must be tdx 

____________________________

   #####   ##   #####    ##
  ###     ###  ##       ###
   ###   # ## ##  #### # ##
    ### ##### ##    # #####
 ##### #   ##  ##### #   ##
____________________________

SAGA Version: 8.2.2

____________________________
library path: /usr/lib/x86_64-linux-gnu/saga/
library name: libstatistics_regression
library     : statistics_regression
tool        : GWR for Grid Downscaling
identifier  : 14
author      : O.Conrad (c) 2013
processors  : 56 [56]
____________________________

loading: N13E103_edem_egm

100%
loading: N13E103_gdtm_riofill0_egm

 99%
[GWR for Grid Downscaling] Execution started...

__________
[GWR for Grid Downscaling] Parameters:

Grid System: 0.000111; 9001x 9001y; 103x 13y
Predictors: 1 object (N13E103_edem_egm)
Regression: Regression
Regression with Residual Correction: <not set>
Dependent Variable: N13E103_gdtm_riofill0_egm
Coefficient of Determination: Coefficient of Determination
Residuals: Residuals
Regression Parameters: No objects
L

Segmentation fault (core dumped)


Input file size is 9001, 9001
0...10...20...30...40...50...60...70...80...90...100 - done.
# Successfully converted "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs.tif" to "/home/ljp238/Downloads/SAGA_DEV/N13E103_GWRd_svs_dw3.tif".
GWR Grid Downscaling completed.
Cleaning up intermediate files...
/home/ljp238/Downloads/SAGA_DEV
Start time: 2025-05-15 11:44:40
End time: 2025-05-15 11:45:43
RUN.TIME = 1 minutes, 3 seconds


##### M-VAR

##### S-VARe

##### M-VARe

In [1]:
from s_downxcale import bcor_sub

In [2]:
# i need egm08 grid, gdtm_egm, edem_egm as block 

geoid_fn = "/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/edem_egm/TLS/EGM08c.tif"
edemH_fn = "/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/edem_egm/TLS/TLS_edem_egm.tif"
gdemh_fn = "/media/ljp238/12TBWolf/BRCHIEVE/GDEM/BLOCK/edem_egm/TLS/gdtmf_0.tif" 

gdemH_fn = gdemh_fn.replace('.tif', '_egm.tif') # 4326 to 4979 gdemh_fn - geoid_fn
bcor_sub(fine_path=gdemh_fn, coarse_path=geoid_fn, output_path=gdemH_fn)